# Auditing a CNN trained on CIFAR10 using the Population Attack

## Introduction

In this tutorial, we will see:

- How to specify the dataset and model for Privacy Meter
- How to audit a Tensorflow model
- How to use the `PopulationMetric` to evaluate membership leakage using loss values from the target model

## Imports

In [1]:
import numpy as np
import tensorflow as tf

For now we install the Privacy Meter library from the local source. A version will be pushed to pip soon.

In [2]:
import sys
!{sys.executable} -m pip install -e ../.
from privacy_meter.dataset import Dataset
from privacy_meter.model import TensorflowModel
from privacy_meter.information_source import InformationSource
from privacy_meter.audit import Audit, MetricEnum

Obtaining file:///Users/aadyaamaddi/Desktop/ML%20Privacy%20Meter/privacy_meter
  Attempting uninstall: privacy-meter
    Found existing installation: privacy-meter 1.0
    Uninstalling privacy-meter-1.0:
      Successfully uninstalled privacy-meter-1.0
  Running setup.py develop for privacy-meter


## Settings

Hyperparameters:

In [3]:
# for training the target model
num_train_points = 5000
num_test_points = 5000
loss_fn = tf.keras.losses.CategoricalCrossentropy()
optim_fn = 'adam'
epochs = 25
batch_size = 64
regularizer_penalty = 0.01
regularizer = tf.keras.regularizers.l2(l=regularizer_penalty)

In [4]:
# for the population metric
num_population_points = 10000
fpr_tolerance_list = [
    0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0
]

## Dataset creation

We use the CIFAR10 dataset for this tutorial. As Tensorflow already has the data loading code for CIFAR10, we just need to add our pre-processing code on top of it.

In [5]:
def preprocess_cifar10_dataset():
    input_shape, num_classes = (32, 32, 3), 10

    # split the data between train and test sets
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

    # scale images to the [0, 1] range
    x_train = x_train.astype("float32") / 255
    x_test = x_test.astype("float32") / 255

    # convert labels into one hot vectors
    y_train = tf.keras.utils.to_categorical(y_train, num_classes)
    y_test = tf.keras.utils.to_categorical(y_test, num_classes)

    return x_train, y_train, x_test, y_test, input_shape, num_classes

x_train_all, y_train_all, x_test_all, y_test_all, input_shape, num_classes = preprocess_cifar10_dataset()

CIFAR10 comes with the predetermined train and test partitions. We further split the train partition into two more sets - 'train' and 'population' for the audit. 

We will have the following sets at the end of this partitioning:

- The 'train' set will be used to train the target model. It will be used as the 'member' set for the audit.
- The 'test' set will be used as the 'non-member' set for the audit.
- The 'population' set will be used as the reference data by the `PopulationMetric`.

In [6]:
x_train, y_train = x_train_all[:num_train_points], y_train_all[:num_train_points]
x_test, y_test = x_test_all[:num_test_points], y_test_all[:num_test_points]
x_population = x_train_all[num_train_points:(num_train_points + num_population_points)]
y_population = y_train_all[num_train_points:(num_train_points + num_population_points)]

We wrap the sets into a `Dataset` object, which takes in the following arguments:

- `data_dict` contains the actual dataset, in the form of a 2D dictionary. The first key corresponds to the split name (here we have two: "train" and "test"), and the second key to the feature name (here we also have two: "x" and "y").
- `default_input` contains the name of the feature that should be used as the models input (here "x").
- `default_output` contains the name of the feature that should be used as the label / models output (here "y").

In [7]:
# create the target model's dataset
train_ds = {'x': x_train, 'y': y_train}
test_ds = {'x': x_test, 'y': y_test}
target_dataset = Dataset(
    data_dict={'train': train_ds, 'test': test_ds},
    default_input='x', default_output='y'
)

# create the reference dataset 
population_ds = {'x': x_population, 'y': y_population}
reference_dataset = Dataset(
    # this is the default mapping that a Metric will look for
    # in a reference dataset
    data_dict={'train': population_ds},
    default_input='x', default_output='y'
)

## Training the target model

We define the Tensorflow model to be used as the target model:

In [8]:
def get_tensorflow_cnn_classifier(input_shape, num_classes, regularizer):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu',
                                     input_shape=input_shape, kernel_regularizer=regularizer))
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu',
                                     kernel_regularizer=regularizer))
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))
    return model

And we compile and train the target model using the target dataset we defined above:

In [9]:
x = target_dataset.get_feature('train', '<default_input>')
y = target_dataset.get_feature('train', '<default_output>')
model = get_tensorflow_cnn_classifier(input_shape, num_classes, regularizer)
model.summary()
model.compile(optimizer=optim_fn, loss=loss_fn, metrics=['accuracy'])
model.fit(x, y, batch_size=batch_size, epochs=epochs, verbose=2)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 2304)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2304)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                2

We wrap the trained target model in a `TensorflowModel` object. Note that we need to pass a loss function to this object.

In [10]:
target_model = TensorflowModel(model_obj=model, loss_fn=loss_fn)

## Information sources

We can now define two `InformationSource` objects. Basically, an information source is an abstraction representing a set of models, and their corresponding datasets. Note that for the `PopulationMetric` we use the same model in both the target and reference information sources, but the datasets that will be queried will differ.

In [11]:
target_info_source = InformationSource(
    models=[target_model], 
    datasets=[target_dataset]
)

reference_info_source = InformationSource(
    models=[target_model],
    datasets=[reference_dataset]
)

## Metric and Audit

We now create a `Metric` object, which is an abstraction representing an algorithm used to measure something on an `InformationSource`, such as membership information leakage. In this case, we use the `PopulationMetric` to measure the membership information leakage of `target_info_source` in a black-box setting, using loss values returned by the target model on the `reference_info_source`.

The `Audit` object is a wrapper to actually run the audit, and display the results. More visualization options will be added soon.

As we will be using the default version of the `PopulationMetric`, we pass the `POPULATION` enum value as the metric argument for the `Audit` object.

In [12]:
audit_obj = Audit(
    metric=MetricEnum.POPULATION,
    target_info_source=target_info_source,
    reference_info_source=reference_info_source,
    fpr_tolerance_list=fpr_tolerance_list
)
audit_obj.prepare()
audit_results = audit_obj.run()
for result in audit_results:
    print(result)

============= METRIC RESULT OBJECT =============
Accuracy          = 0.5001
ROC AUC Score     = 0.5001
FPR               = 0.0
TN, FP, FN, TP    = (5000, 0, 4999, 1)
============= METRIC RESULT OBJECT =============
Accuracy          = 0.5089
ROC AUC Score     = 0.5089
FPR               = 0.1072
TN, FP, FN, TP    = (4464, 536, 4375, 625)
============= METRIC RESULT OBJECT =============
Accuracy          = 0.5209
ROC AUC Score     = 0.5209
FPR               = 0.2106
TN, FP, FN, TP    = (3947, 1053, 3738, 1262)
============= METRIC RESULT OBJECT =============
Accuracy          = 0.5266
ROC AUC Score     = 0.5266000000000001
FPR               = 0.3126
TN, FP, FN, TP    = (3437, 1563, 3171, 1829)
============= METRIC RESULT OBJECT =============
Accuracy          = 0.5349
ROC AUC Score     = 0.5349
FPR               = 0.4094
TN, FP, FN, TP    = (2953, 2047, 2604, 2396)
============= METRIC RESULT OBJECT =============
Accuracy          = 0.5367
ROC AUC Score     = 0.5367000000000001
FPR      